# News Title Classification
---

<img src="assets/news.jpg" width=70%>

In this notebook, we will classify news into categories based on their headline. Our goal is to identify whether a headline is belong to one of four classes: 
- Entertainment
- Business
- Technology
- Medical

To achieve that, we will use a recurrent neural network (RNN). Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. We will use multiple bidirectional GRU/LSTM layers in the network. The bidirectional LSTM/GRU keeps the contextual information in both directions which is pretty useful in text classification task. We also use attention model to build our model architecture. 

In the past conventional methods like TFIDF/CountVectorizer etc., we used to find features from text by doing a keyword extraction. Some word are more helpful in determining the category of a text than others. But in this method we sort of lost the sequential structure of text. With LSTM and deep learning methods while we are able to take case of the sequence structure we lose the ability to give higher weightage to more important words. 

Additionally, attention mechanism is introduced to extract such words that are important to the meaning of the sentence and aggregate the representation of those informative words to form a sentence vector.

The following workflow is performed:
1. **Data Import**: Data is the lifeblood of predictive analytics. We have to know which data to use, where to gather them, and how to make them useful to solve our problem. 
2. **Data Cleaning**: Raw data are generally incomplete, inconsistent, and contain many errors. Thus, we need to prepare the data for further processing. 
3. **Data Pre-Processing**: We structure and enrich data into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes, such as analytics. 
4. **Data Modelling**: We will use a recurrent neural network (RNN).
5. **Prediction**: Once we have developed the best predictive model, we can deploy it to make predictions. 

Before moving to the next section, we need to import all packages required to do the analysis by calling the following:

In [1]:
# Data analysis packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Deep learning packages
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import TensorDataset, DataLoader

# Miscellaneous
import bcolz
import pickle
import re

### Table of Contents
- [**1.0. Import the Data**](#section 1.0.)


- [**2.0. Clean the Data**](#section 2.0.)
  - [2.1. Lowercase All the Words](#section 2.1.)
  - [2.2. Remove Short Forms](#section 2.2.)
  - [2.3. Remove Punctuations and Symbols](#section 2.3.)


- [**3.0. Data Pre-Processing**](#section 3.0.)
  - [3.1. Tokenize the Titles](#section 3.1.)
  - [3.2. Track Vocabulary](#section 3.2.)
  - [3.3. Encode the Data](#section 3.3.)
  - [3.4. Pad the Titles](#section 3.4.)
  
  
- [**4.0. Define Training, Validation, and Test Set**](#section 4.0.)


- [**5.0. Build Network Architecture**](#section 5.0.)
  - [5.1. Import Pre-Trained Word Embeddings](#section 5.1.)
  - [5.2. Define RNN Architecture](#section 5.2.)
  - [5.3. Instantiate the network](#section 5.3.)
  - [5.4. Training](#section 5.4.)


- [**6.0. Prediction**](#section 6.0.)

---
<a id='section 1.0.'></a>
## 1.0. Import the Data

In [2]:
# Import dataset
df = pd.read_excel('data/News Title.xls')

# Show the first 5 rows the dataset
df.head()

,No,News Title,Category
0,1,Google+ rolls out 'Stories' for tricked out ph...,Technology
1,2,Dov Charney's Redeeming Quality,Business
2,3,White God adds Un Certain Regard to the Palm Dog,Entertainment
3,4,"Google shows off Androids for wearables, cars,...",Technology
4,5,China May new bank loans at 870.8 bln yuan,Business


We only need data from `News Title` and `Category` column. So, we put them in the `titles` and `labels` variable, respectively. 

In [3]:
titles = df['News Title']
labels = df['Category']

Let's see the distribution of labels in the dataset.

In [4]:
labels.value_counts()

Entertainment    23961
Business         17707
Technology       16776
Medical           7091
Name: Category, dtype: int64

To enable analytics, we transform the label into integers by calling the following. 

In [5]:
# Replace labels with integers
dict_labels = {'Entertainment':0, 'Business':1, 'Technology': 2, 'Medical':3}
labels = pd.Series(labels).replace(dict_labels)

---
<a id='section 2.0.'></a>
## 2.0. Clean the Data
To improve the predictive performance of the model, the following data cleaning are performed:
- Lowercase all the words in the data
- Remove short forms
- Remove symbols and punctuations

<a id='section 2.1.'></a>
### 2.1. Lowercase All the Words
Sentences can contain a mixture of uppercase and lower case letters. Multiple sentences make up a text document. To reduce the problem space, the most common approach is to reduce everything to lower case.

In [6]:
# Lowercase all words
titles = titles.apply(lambda x: x.lower())

<a id='section 2.2.'></a>
### 2.2. Remove Short Forms
There are some short form words in the dataset. We will fix this.

In [7]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

We define `clean_shortforms` function to fix short forms.

In [8]:
def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
titles = titles.apply(lambda x: clean_shortforms(x))

<a id='section 2.3.'></a>
### 2.3. Remove Punctuations and Symbols
We will remove all punctuations and symbols from the dataset using `clean_symbol` function. 

In [9]:
symbols = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

In [10]:
def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, '')
    return text

# remove symbols and punctuations 
titles = titles.apply(lambda x: clean_symbol(x))

---
<a id='section 3.0.'></a>
## 3.0. Data Pre-Processing
We will implement the following pre-processing functions:
- Tokenize the titles
- Track vocabulary
- Encode the data
- Pad the titles

<a id='section 3.1.'></a>
### 3.1. Tokenize the Titles
We will be splitting the titles into a word array using spaces as delimiters.

In [11]:
# tokenize all titles in the data
titles_token = titles.apply(lambda x: x.split())

In [12]:
print('Average word length of titles is {0:.0f}.'.format(np.mean(titles_token.apply(lambda x: len(x)))))
print('Max word length of titles is {0:.0f}.'.format(np.max(titles_token.apply(lambda x: len(x)))))
print('Min word length of titles is {0:.0f}.'.format(np.min(titles_token.apply(lambda x: len(x)))))

Average word length of titles is 9.
Max word length of titles is 1819.
Min word length of titles is 1.


In [13]:
# Distribution of word length of titles
count = [len(title) for title in titles_token]
pd.Series(count).value_counts()

9       9331
8       9075
10      8807
11      7757
7       7654
12      5782
6       5481
13      3466
5       3229
14      1753
4       1400
15       694
3        558
2        255
16       201
17        60
1         22
18         6
71         1
1784       1
19         1
1819       1
dtype: int64

As we can see, there are a few titles that have more than 100 words. This is quite odd for a title. We consider these titles to be outliers. Thus, we will exclude all titles that have more than 100 words. 

In [14]:
# Exclude all titles that have more than 100 words
titles_token = [title for title in titles_token if len(title) < 100]
labels = np.array([label for title, label in zip(titles_token, labels) if len(title) < 100])

<a id='section 3.2.'></a>
### 3.2. Track Vocabulary
We define `track_vocab` to track our training vocabulary, which goes through all our text and counts the occurence of the contained words.

In [15]:
def track_vocab(sentences, verbose =  True):
    
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
                
    return vocab

In [16]:
# count the occurrence of all words in the data
vocab_count = track_vocab(titles_token)
print({k: vocab_count[k] for k in list(vocab_count)[:5]})

{'google': 1683, 'rolls': 62, 'out': 1029, 'stories': 67, 'for': 8133}


<a id='section 3.3.'></a>
### 3.3. Encode the Data
Since we will use embedding layers, we'll need to encode each word with an integer. To create a word embedding, we first need to transform the words to ids.  In this function, we create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

We return these dictionaries in the **tuple** `(vocab_to_int, int_to_vocab)`

In [17]:
def create_lookup_tables(vocab_count):
    
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
    # create vocab_to_int dictionary
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(vocab_count)

Then, we can convert our data into integers, so they can be passed into the network.

In [18]:
# encode the data
title_ints = []
for title in titles_token:
    title_ints.append([vocab_to_int[word] for word in title])

<a id='section 3.4.'></a>
### 3.4. Pad the Titles
To deal with both short and very long title, we'll pad or truncate all our titles to a specific length. For titles shorter than some `seq_length`, we'll pad with 0s. For titles longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 31, because the maximum title length from the data is 31.  

In [19]:
def pad_features(sentences_token, seq_length):
    # getting the correct rows x cols shape
    features = np.zeros((len(sentences_token), seq_length), dtype=int)

    # for each title, I grab that title and 
    for i, row in enumerate(sentences_token):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

# pad the titles
seq_length = 31
features = pad_features(title_ints, seq_length)

---
<a id='section 4.0.'></a>
## 4.0. Define Training, Validation, and Test Set
With our data in nice shape, we'll split it into training, validation, and test sets.

In [20]:
train_X, val_test_X, train_y, val_test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)

val_X, test_X, val_y, test_y = train_test_split(val_test_X, val_test_y, 
                                                test_size=0.5, 
                                                random_state=42, shuffle=True,
                                                stratify=val_test_y)

### DataLoaders and Batching
After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

In [21]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_X), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))

# dataloaders
batch_size = 50
num_workers = 8

# make sure to SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=num_workers)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, num_workers=num_workers)

In [22]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 31])
Sample input: 
 tensor([[    0,     0,     0,  ..., 25034,   900,  3322],
        [    0,     0,     0,  ...,     5,   425,  3243],
        [    0,     0,     0,  ...,  4634,     0,   136],
        ...,
        [    0,     0,     0,  ...,   143,   278,  4303],
        [    0,     0,     0,  ...,    78,   413,  1204],
        [    0,     0,     0,  ...,  1182,     6,  6458]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 1, 3, 3, 1, 2, 0, 0, 0, 1, 0, 0, 2, 3, 1, 0, 0, 1, 2, 1, 1, 3, 3,
        0, 3, 0, 1, 1, 1, 0, 0, 2, 0, 1, 0, 2, 3, 2, 2, 1, 3, 0, 3, 2, 1, 1, 1,
        0, 3])


<a id='section 5.0.'></a>
## 5.0. Build Network Architecture
<a id='section 5.1.'></a>
### 5.1. Import Pre-Trained Word Embeddings
**Note**: This part is taken from a Medium article [How to use Pre-trained Word Embeddings in PyTorch](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76). 

Rather than training our own word vectors from scratch for word embedding, we will leverage on GloVe. Its authors have released four text files with word vectors trained on different massive web datasets. They are available for download [here](https://nlp.stanford.edu/projects/glove/). We will use “Wikipedia 2014 + Gigaword 5” which is the smallest file (“ [glove.6B.zip](http://nlp.stanford.edu/data/wordvecs/glove.6B.zip)”) with 822 MB. It was trained on a corpus of 6 billion tokens and contains a vocabulary of 400 thousand tokens.

We need to parse the file to get as output: list of words, dictionary mapping each word to their id (position) and array of vectors. Given that the vocabulary have 400k tokens, we will use [bcolz](https://github.com/Blosc/bcolz) to store the array of vectors. It provides columnar, chunked data containers that can be compressed either in-memory and on-disk. It is based on NumPy, and uses it as the standard data container to communicate with bcolz objects. We then save the outputs to disk for future uses.

In [23]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'embedding/glove/6B.300.dat', mode='w')

with open(f'embedding/glove/glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 300)), rootdir=f'embedding/glove/6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'embedding/glove/6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'embedding/glove/6B.300_idx.pkl', 'wb'))

Using those objects we can now create a dictionary that given a word returns its vector.

In [24]:
vectors = bcolz.open(f'embedding/glove/6B.300.dat')[:]
words = pickle.load(open(f'embedding/glove/6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'embedding/glove/6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

What we need to do at this point is to create an embedding layer, that is a dictionary mapping integer indices (that represent words) to dense vectors. It takes as input integers, it looks up these integers into an internal dictionary, and it returns the associated vectors.

We have already built a Python dictionary with similar characteristics, but it does not support auto differentiation so can not be used as a neural network layer and was also built based on GloVe’s vocabulary, likely different from our dataset’s vocabulary. In PyTorch an embedding layer is available through torch.nn.Embedding class.

We must build a matrix of weights that will be loaded into the PyTorch embedding layer. Its shape will be equal to (dataset’s vocabulary length, word vectors dimension).

For each word in dataset’s vocabulary, we check if it is on GloVe’s vocabulary. If it do it, we load its pre-trained word vector. Otherwise, we initialize a random vector.

In [39]:
sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
target_vocab = sorted_vocab
emb_dim = 300

matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, emb_dim))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

In [40]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

<a id='section 5.2.'></a>
### 5.2. Define RNN Architecture

In this model, we use multiple bidirectional GRU/LSTM layers in the network. The bidirectional LSTM/GRU keeps the contextual information in both directions which is pretty useful in text classification task. 

We also use attention model to build our model architecture. In the past conventional methods like TFIDF/CountVectorizer etc., we used to find features from text by doing a keyword extraction. Some word are more helpful in determining the category of a text than others. But in this method we sort of lost the sequential structure of text. With LSTM and deep learning methods while we are able to take case of the sequence structure we lose the ability to give higher weightage to more important words. Attention mechanism is introduced to extract such words that are important to the meaning of the sentence and aggregate the representation of those informative words to form a sentence vector.

In [41]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [42]:
# implementation of attention layer
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens to hundreds of thousands of words, so we will need a more efficient representation for our input data than one-hot encoded vectors. 

>**After input words are passed to an embedding layer, the new embeddings will be passed to bidirectional LSTM/GRU layers.** These layers will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in our data. The bidirectional LSTM/GRU layers keep the contextual information in both directions which is pretty useful in text classification task. 

>**The outputs of the bidirectional LSTM/GRU layers will be passed to the attention layer.** Attention mechanism is introduced to extract such words that are important to the meaning of the sentence and aggregate the representation of those informative words to form a sentence vector.

>**The outputs of the attention layer will be passed to the second bidirectional LSTM/GRU layers.** 

>**Finally, the outputs will go to a output layer.** We are using a fully-connected neural network layer.

In [43]:
import torch.nn as nn

class RNN(nn.Module):
    """
    The RNN model that will be used to perform classification.
    """

    def __init__(self, weights_matrix, output_size, hidden_dim, drop_prob=0.1):
        """
        Initialize the model by setting up the layers.
        """
        super(RNN, self).__init__()

        self.output_size = output_size
        self.hidden_dim = hidden_dim
        
        # embedding layers
        self.embedding, self.num_embeddings, self.embedding_dim = create_emb_layer(weights_matrix, True)
        
        # embedding dropout
        self.dropout = nn.Dropout2d(drop_prob)
        
        # First lstm and GRU layers
        self.lstm1 = nn.LSTM(self.embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.gru1 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # attention layer
        self.attention = Attention(hidden_dim*2, seq_length)
        
        # Second lstm and GRU layers
        self.lstm2 = nn.LSTM(hidden_dim * 2, hidden_dim, batch_first=True, bidirectional=True)
        self.gru2 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # linear
        self.fc = nn.Linear(hidden_dim * 2, 64)
        self.out = nn.Linear(64, self.output_size)
        
        self.relu = nn.ReLU()
        

    def forward(self, x):
        """
        Perform a forward pass of our model on some inputs.
        """
        batch_size = x.size(0)

        # embedding output
        x = x.long()
        embeds = self.embedding(x)
        embeds = torch.squeeze(torch.unsqueeze(embeds, 0))
        
        # lstm, gru, and attention outputs
        lstm_out, _ = self.lstm1(embeds)
        gru_out, _ = self.gru1(lstm_out)
        attention_out = self.attention(gru_out, 256)
        attention_out = attention_out.view(batch_size, -1, self.hidden_dim * 2)
        lstm_out, _ = self.lstm2(attention_out)
        gru_out, _ = self.gru2(lstm_out)
        
        # linear outputs
        out = gru_out.view(-1, gru_out.shape[2])
        fc_out = self.relu(self.fc(out))
        final_out = self.out(fc_out)
    
        return final_out

<a id='section 5.3.'></a>
### 5.3. Instantiate the network
Here, we'll instantiate the network. First up, defining the hyperparameters.

* `weights_matrix`: The pre-trained word vector.
* `output_size`: Size of our desired output.
* `hidden_dim`: Number of units in the hidden layers of our LSTM/GRU cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM/GRU layers in the network. Typically between 1-3. We use the value of 1 as our `n_layers`.

In [44]:
# Instantiate the model w/ hyperparams
weights_matrix = weights_matrix
output_size = 4
hidden_dim = 256

net = RNN(weights_matrix, output_size, hidden_dim)

print(net)

RNN(
  (embedding): Embedding(32779, 300)
  (dropout): Dropout2d(p=0.1)
  (lstm1): LSTM(300, 256, batch_first=True, bidirectional=True)
  (gru1): GRU(512, 256, batch_first=True, bidirectional=True)
  (attention): Attention()
  (lstm2): LSTM(512, 256, batch_first=True, bidirectional=True)
  (gru2): GRU(512, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=4, bias=True)
  (relu): ReLU()
)


<a id='section 5.4.'></a>
### 5.4. Training
We use `CrossEntropyLoss` as our loss function and `SGD` as our optimizer. We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [48]:
# loss and optimization functions
lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)

In [50]:
# training params

epochs = 1

counter = 0
print_every = 100
clip = 5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output = net(inputs)

        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output = net(inputs)
                val_loss = criterion(output, labels)

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/1... Step: 100... Loss: 0.510917... Val Loss: 0.723591
Epoch: 1/1... Step: 200... Loss: 0.633727... Val Loss: 0.718747
Epoch: 1/1... Step: 300... Loss: 0.679442... Val Loss: 0.721072
Epoch: 1/1... Step: 400... Loss: 0.530738... Val Loss: 0.708940
Epoch: 1/1... Step: 500... Loss: 0.563486... Val Loss: 0.709165
Epoch: 1/1... Step: 600... Loss: 0.548085... Val Loss: 0.706518
Epoch: 1/1... Step: 700... Loss: 0.503572... Val Loss: 0.709962
Epoch: 1/1... Step: 800... Loss: 0.409615... Val Loss: 0.711098
Epoch: 1/1... Step: 900... Loss: 0.793967... Val Loss: 0.709682
Epoch: 1/1... Step: 1000... Loss: 0.969674... Val Loss: 0.702137


<a id='section 6.0.'></a>
## 6.0. Predictions
We'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

In [51]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output = net(inputs)
    
    # calculate loss
    test_loss = criterion(output, labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0, 1, 2, or 3)
    pred = output.data.max(1, keepdim=True)[1]  
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.684
Test accuracy: 0.788


Our mode is able to achieve **78.8% accuracy**. 

#### Save the Model
After running the above training cell, the trained model will be saved by name, `trained_rnn`.

In [52]:
import os

def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)

In [53]:
# saving the trained model
save_model('./save/trained_rnn', net)
print('Model Trained and Saved')

Model Trained and Saved


/anaconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/anaconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
